<a href="https://colab.research.google.com/github/lgolab/Fine-tuning-data-dependencies/blob/main/MFD_Tableaux_FlightDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data Preparation

### Importing Libraries

In [1]:
import csv
from datetime import datetime
from tqdm import tqdm
from itertools import chain, combinations

### Moving data to colab

Please replace this part of code with downloading the data (including `August2018-Nationwide-flights.csv` and `AirCarriers`) and putting it in the `/content/flight` folder)

In [3]:
!cp -r /content/drive/MyDrive/FD-project/flight/ /content/flight
%cd /content/flight/

/content/flight


### Reading the data and Creating the dataset

In [9]:
dataset_address = 'August2018-Nationwide-flights.csv'

attrs = ['FL_DATE', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM',
         'ORIGIN', 'DEST',
         'ORIGIN_CITY_MARKET_ID', 'DEST_CITY_MARKET_ID', 
         'DEP_DELAY', 'ARR_DELAY']
attrIndex = []

dataset = []
with open(dataset_address) as csvfile:    
    reader = csv.reader(csvfile, delimiter=',')
    
    cnt = 0
    
    for row in reader:
        if cnt == 0:
            for attr in attrs:
                attrIndex.append(row.index(attr))
        else:
            tupple = {}
            for i in range(len(attrs)):
                tupple[attrs[i]] = row[attrIndex[i]]
            
            dataset.append(tupple)
                
        cnt += 1
    

print('Number of tuples in dataset: ', len(dataset), '\nA sample tuple:\n', dataset[5])


Number of tuples in dataset:  701352 
A sample tuple:
 {'FL_DATE': '2018-08-01', 'OP_CARRIER_AIRLINE_ID': '19805', 'OP_CARRIER_FL_NUM': '1594', 'ORIGIN': 'DTW', 'DEST': 'ORD', 'ORIGIN_CITY_MARKET_ID': '31295', 'DEST_CITY_MARKET_ID': '30977', 'DEP_DELAY': '-10.00', 'ARR_DELAY': '-7.00'}


### Filtering the dataset

In [10]:
US_major_airports = ['ATL', 'BOS', 'DEN', 'ORD', 'LAX', 'CLT', 'LAS', 'PHX', 'JFK', 'SEA']
filter_dic = {
        'ORIGIN': US_major_airports,    # 10/many!
        'DEST': US_major_airports    # 10/many!
}

filtered_dataset = []
for tupple in dataset:
    filter_flag = 0
    for key in filter_dic:
        if tupple[key] not in filter_dic[key]:
            filter_flag = 1
    if not filter_flag:
        filtered_dataset.append(tupple)

print('Dataset size before filtering: ', len(dataset))
print('Dataset size after filtering: ', len(filtered_dataset))
dataset = filtered_dataset

Dataset size before filtering:  701352
Dataset size after filtering:  43266


### Making fields human-readable

In [11]:
AirlineMap = {}
with open('AirCarriers') as csvfile:    
    reader = csv.reader(csvfile, delimiter=',')
    cnt = 0
    
    for row in reader:
        if cnt != 0:
            AirlineMap[row[0]] = row[1]
        cnt += 1

for i in range(len(dataset)):
    dataset[i]['OP_CARRIER_AIRLINE_ID'] = AirlineMap[dataset[i]['OP_CARRIER_AIRLINE_ID']]

# Test:
print(len(dataset))
print(dataset[0])

43266
{'FL_DATE': '2018-08-01', 'OP_CARRIER_AIRLINE_ID': 'American Airlines Inc.: AA', 'OP_CARRIER_FL_NUM': '1587', 'ORIGIN': 'JFK', 'DEST': 'PHX', 'ORIGIN_CITY_MARKET_ID': '31703', 'DEST_CITY_MARKET_ID': '30466', 'DEP_DELAY': '9.00', 'ARR_DELAY': '44.00'}


# 2. Implementing CWCG algorithm

The codes in this part can be applied on other datasets with the similar python dictionary format. 

### Function for adding important patterns

In [12]:
def add_patterns(general_patterns):
    global patterns, pattern_members, entry_patterns

    pattern_tuples = {}
    
    for entry_index, entry in enumerate(dataset):
        pattern_tuple = tuple([])
        pattern_dictionary = {}
        
        for attr in general_patterns:
            pattern_tuple = pattern_tuple + tuple([entry[attr]])
            pattern_dictionary[attr] = entry[attr]
        
        if pattern_tuple not in pattern_tuples:
            pattern_tuples[pattern_tuple] = len(patterns)
            patterns.append(pattern_dictionary)
            pattern_members.append([])
        
        pattern_index = pattern_tuples[pattern_tuple]
        pattern_members[pattern_index].append(entry_index)
        entry_patterns[entry_index].append(pattern_index)


### Functions for setting cost of patterns

In [13]:
def calculate_delta(arr, confidence = 0.9):
    n = len(arr)
    removals = n - round(n * confidence)
    best_delta = arr[-1] - arr[0]

    # removing outliers in such a way that minimize delta:
    for i in range(removals+1):
        best_delta = min(best_delta, arr[-(removals-i+1)] - arr[i])
    
    return best_delta

In [14]:
def calculate_costs(MFD_LHS_attrs, MFD_target, base = 2, confidence1 = 0.9, confidence2 = 1):
    global pattern_costs, pattern_deltas
    pattern_costs = [0 for i in range(len(patterns))]
    pattern_deltas = [0 for i in range(len(patterns))]

    inf = 1e8
    eps = 0.5

    # Calculating fixed-antecedent group deltas:

    global group_deltas, group_members
    group_deltas = {}
    group_members = {}

    for entry_index, entry in enumerate(dataset):
        if not dataset[entry_index][MFD_target].replace('.','',1).replace('-','',1).isnumeric():
            continue

        group_tuple = tuple([])
    
        for attr in MFD_LHS_attrs:
            group_tuple = group_tuple + tuple([entry[attr]])
        
        if group_tuple not in group_members:
            group_members[group_tuple] = []
        group_members[group_tuple].append(float(dataset[entry_index][MFD_target]))
    
    for group_tuple in group_members:
        arr = group_members[group_tuple]
        arr.sort()
        group_deltas[group_tuple] = calculate_delta(arr, confidence=confidence1)


    # Calculating pattern deltas: 

    for i, pattern in enumerate(patterns):

        pattern_groups = set([])

        for entry_index in pattern_members[i]:
            if dataset[entry_index][MFD_target].replace('.','',1).replace('-','',1).isnumeric():
                group_tuple = tuple([])

                for attr in MFD_LHS_attrs:
                    group_tuple = group_tuple + tuple([dataset[entry_index][attr]])

                pattern_groups.add(group_tuple)

                
        if len(pattern_groups) == 0:
            pattern_costs[i] = inf
        else:
            deltas = []
            for group_tuple in pattern_groups:
                deltas.append(group_deltas[group_tuple])
            
            deltas.sort()
            n = len(deltas)
            overall_delta = deltas[round(n * confidence2)-1] 

            pattern_deltas[i] = overall_delta

            pattern_costs[i] = (base ** (overall_delta/15))


### CWSC function

In [16]:
# In this function, we use dataset, patterns, pattern_members, entry_patterns, and patterns as global variables

def CWSC(k, coverage):
    
    total_cost = 0

    # Moving pattern_members to members
    members = [set([]) for i in range(len(patterns))]
    for i in range(len(patterns)):
        for member in pattern_members[i]:
            members[i].add(member)
    
    selected = [0 for i in range(len(patterns))]
    solution_patterns = []
    rem = int(len(dataset) * coverage)
    
    for i in range(k, 0, -1):
        
        max_gain = 0.0
        max_pattern_index = -1
        
        for index, pattern in enumerate(patterns):
            
            marginal_benefit = len(members[index])
            cost = pattern_costs[index]
            marginal_gain = marginal_benefit / cost
            
            if selected[index] == 0 and marginal_benefit >= rem/i and marginal_gain > max_gain:
                max_gain = marginal_gain
                max_pattern_index = index
        
        if max_pattern_index == -1:
            print('Impossible!')
            return ['Failed!']
        
        # selecting the next pattern
        selected[max_pattern_index] = 1
        total_cost += pattern_costs[max_pattern_index]
        solution_patterns.append(max_pattern_index)
        rem -= len(members[max_pattern_index])

        if rem <= 0:
            print('\nTotal cost is ', total_cost, '\n')
            return solution_patterns, total_cost
        
        # updating the marginal benefits
        for entry_index in members[max_pattern_index]:
            for pattern_index in entry_patterns[entry_index]:
                if pattern_index != max_pattern_index:
                    members[pattern_index].remove(entry_index)


### Running the algorithm on the dataset

In [17]:
import timeit

def generate_MFD_tableaux(MFD_LHS_attrs, MFD_target, k = 20, confidence = 0.9, coverage = 0.5):

    # Adding patterns:
    print('\nAdding all the patterns... \n')
    t1 = timeit.default_timer()

    global patterns, pattern_members, entry_patterns
    patterns = []
    pattern_members = []
    entry_patterns = [[] for i in range(len(dataset))]

    for x in chain.from_iterable(combinations(MFD_LHS_attrs, r) for r in range(0, len(MFD_LHS_attrs)+1)):
        general_pattern = list(x)
        print(general_pattern)
        add_patterns(general_pattern)

    t2 = timeit.default_timer()
    print('\nAdding patterns finished!\n')
    print('Time elapsed: ', t2-t1)

    print('Number of patterns: ', len(patterns))
    
    # Calculating pattern costs:
    print('\nCalculating pattern costs...\n')
    calculate_costs(MFD_LHS_attrs, MFD_target, confidence1=confidence)

    t3 = timeit.default_timer()
    print('\nCalculating pattern costs finished!\n')
    print('Time elapsed: ', t3-t2)

    #Running CWSC algorithm:
    print('\nRunning CWSC algorithm...\n')
    solution_patterns, total_cost = CWSC(k, coverage)

    t4 = timeit.default_timer()
    print('Time elapsed: ', t4-t3, '\n')
    print('\nTotal elapsed time: ', t4-t1, '\n')

    print('\nTableaux:', '(size = ' + str(len(solution_patterns)) + ')\n')
    for pattern_index in solution_patterns:
        print(patterns[pattern_index], 'delta:', pattern_deltas[pattern_index])

    return solution_patterns, total_cost

In [18]:
# Generating MFD tableaux: 
solution_patterns, total_cost = generate_MFD_tableaux(
    MFD_LHS_attrs = ['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST'],
    MFD_target = 'DEP_DELAY', 
    k = 50,
    coverage = 0.6)


Adding all the patterns... 

[]
['OP_CARRIER_AIRLINE_ID']
['OP_CARRIER_FL_NUM']
['ORIGIN']
['DEST']
['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM']
['OP_CARRIER_AIRLINE_ID', 'ORIGIN']
['OP_CARRIER_AIRLINE_ID', 'DEST']
['OP_CARRIER_FL_NUM', 'ORIGIN']
['OP_CARRIER_FL_NUM', 'DEST']
['ORIGIN', 'DEST']
['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'ORIGIN']
['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'DEST']
['OP_CARRIER_AIRLINE_ID', 'ORIGIN', 'DEST']
['OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST']
['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST']

Adding patterns finished!

Time elapsed:  1.5219098739999026
Number of patterns:  21775

Calculating pattern costs...


Calculating pattern costs finished!

Time elapsed:  2.252412906000245

Running CWSC algorithm...


Total cost is  10680.360445154929 

Time elapsed:  0.8655125029999908 


Total elapsed time:  4.639835283000139 


Tableaux: (size = 50)

{'ORIGIN': 'LAX', 'DEST': 'SEA'} delta: 93.0
{'ORIGIN': 'LAS', 'DEST': 'LAX'} delt

# 3. Baseline Algorithm

In [19]:
def set_cover(coverage):
    total_cost = 0

    # Moving pattern_members to members
    members = [set([]) for i in range(len(patterns))]
    for i in range(len(patterns)):
        for member in pattern_members[i]:
            members[i].add(member)
    
    selected = [0 for i in range(len(patterns))]
    solution_patterns = []
    global banned_patterns
    rem = int(len(dataset) * coverage)
    
    while True:
        
        max_support = 0
        max_pattern_index = -1
        
        for index, pattern in enumerate(patterns):
            
            marginal_support = len(members[index])
            
            if banned_patterns[index] == 0 and selected[index] == 0 and marginal_support > max_support:
                max_support = marginal_support
                max_pattern_index = index
        
        if max_pattern_index == -1:
            print('Impossible!')
            return ['Failed!']
        
        # selecting the next pattern
        selected[max_pattern_index] = 1
        total_cost += pattern_costs[max_pattern_index]
        solution_patterns.append(max_pattern_index)
        rem -= len(members[max_pattern_index])

        if rem <= 0:
            print('\nTotal cost is ', total_cost, '\n')
            return solution_patterns, total_cost
        
        # updating the marginal supports
        for entry_index in members[max_pattern_index]:
            for pattern_index in entry_patterns[entry_index]:
                if pattern_index != max_pattern_index:
                    members[pattern_index].remove(entry_index)

In [20]:
import heapq

def optimized_set_cover(coverage):
    total_cost = 0

    # Moving pattern_members to members
    members = [set([]) for i in range(len(patterns))]
    for i in range(len(patterns)):
        for member in pattern_members[i]:
            members[i].add(member)
    
    solution_patterns = []
    global banned_patterns
    rem = int(len(dataset) * coverage)

    remaining_patterns = []
    heapq.heapify(remaining_patterns)
    for index, pattern in enumerate(patterns):
        if banned_patterns[index] == 0:
            heapq.heappush(remaining_patterns, (-len(members[index]), index))
    
    
    while True:
        
        _, max_pattern_index = heapq.heappop(remaining_patterns)

        # selecting the next pattern
        total_cost += pattern_costs[max_pattern_index]
        solution_patterns.append(max_pattern_index)
        rem -= len(members[max_pattern_index])

        if rem <= 0:
            print('\nTotal cost is ', total_cost, '\n')
            return solution_patterns, total_cost
        
        # updating the marginal supports
        for entry_index in members[max_pattern_index]:
            for pattern_index in entry_patterns[entry_index]:
                if pattern_index != max_pattern_index and banned_patterns[pattern_index] == 0:
                    remaining_patterns.pop(remaining_patterns.index((-len(members[pattern_index]), pattern_index)))
                    members[pattern_index].remove(entry_index)
                    heapq.heappush(remaining_patterns, (-len(members[pattern_index]), pattern_index))


In [24]:
def generate_baseline_MFD_tableaux(MFD_LHS_attrs, MFD_target, max_delta, coverage = 0.6):

    # Adding patterns:
    print('\nAdding all the patterns... \n')

    global patterns, pattern_members, entry_patterns 
    patterns = []
    pattern_members = []
    entry_patterns = [[] for i in range(len(dataset))]

    for x in chain.from_iterable(combinations(MFD_LHS_attrs, r) for r in range(0, len(MFD_LHS_attrs)+1)):
        general_pattern = list(x)
        print(general_pattern)
        add_patterns(general_pattern)

    print('\nAdding patterns finished!\n')

    print('Number of patterns: ', len(patterns))

    # Calculating pattern costs:
    print('\nCalculating pattern costs...\n')
    calculate_costs(MFD_LHS_attrs, MFD_target)
    print('\nCalculating pattern costs finished!\n')

    global banned_patterns
    banned_patterns = [0 for i in range(len(patterns))]
    for i in range(len(patterns)):
        if pattern_deltas[i] > max_delta:
            banned_patterns[i] = 1

    #Running set-cover algorithm:
    print('\nRunning set-cover algorithm...\n')
    solution_patterns, total_cost = set_cover(coverage)
    print('\nTableaux:', '(size = ' + str(len(solution_patterns)) + ')\n')

    for pattern_index in solution_patterns:
        support = round(len(pattern_members[pattern_index])/len(dataset)*100, 2)
        print(patterns[pattern_index], '\tdelta:', pattern_deltas[pattern_index], ' support(%):', support)

    return solution_patterns, total_cost

In [25]:
def generate_optimized_baseline_MFD_tableaux(MFD_LHS_attrs, MFD_target, max_delta, coverage = 0.6):

    # Adding patterns:
    print('\nAdding all the patterns... \n')

    global patterns, pattern_members, entry_patterns 
    patterns = []
    pattern_members = []
    entry_patterns = [[] for i in range(len(dataset))]

    for x in chain.from_iterable(combinations(MFD_LHS_attrs, r) for r in range(0, len(MFD_LHS_attrs)+1)):
        general_pattern = list(x)
        print(general_pattern)
        add_patterns(general_pattern)

    print('\nAdding patterns finished!\n')

    print('Number of patterns: ', len(patterns))

    # Calculating pattern costs:
    print('\nCalculating pattern costs...\n')
    calculate_costs(MFD_LHS_attrs, MFD_target)
    print('\nCalculating pattern costs finished!\n')

    global banned_patterns
    banned_patterns = [0 for i in range(len(patterns))]
    for i in range(len(patterns)):
        if pattern_deltas[i] > max_delta:
            banned_patterns[i] = 1

    #Running set-cover algorithm:
    print('\nRunning set-cover algorithm...\n')
    solution_patterns, total_cost = optimized_set_cover(coverage)
    print('\nTableaux:', '(size = ' + str(len(solution_patterns)) + ')\n')

    for pattern_index in solution_patterns:
        support = round(len(pattern_members[pattern_index])/len(dataset)*100, 2)
        print(patterns[pattern_index], '\tdelta:', pattern_deltas[pattern_index], ' support(%):', support)

    return solution_patterns, total_cost

In [26]:
# Gneerate MFD tableaux with the Baseline Algorithm:

baseline_patterns, baseline_cost = generate_optimized_baseline_MFD_tableaux(
    MFD_LHS_attrs = ['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST'],
    MFD_target = 'DEP_DELAY', 
    max_delta = 500,
    coverage = 0.6)


Adding all the patterns... 

[]
['OP_CARRIER_AIRLINE_ID']
['OP_CARRIER_FL_NUM']
['ORIGIN']
['DEST']
['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM']
['OP_CARRIER_AIRLINE_ID', 'ORIGIN']
['OP_CARRIER_AIRLINE_ID', 'DEST']
['OP_CARRIER_FL_NUM', 'ORIGIN']
['OP_CARRIER_FL_NUM', 'DEST']
['ORIGIN', 'DEST']
['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'ORIGIN']
['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'DEST']
['OP_CARRIER_AIRLINE_ID', 'ORIGIN', 'DEST']
['OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST']
['OP_CARRIER_AIRLINE_ID', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST']

Adding patterns finished!

Number of patterns:  21775

Calculating pattern costs...


Calculating pattern costs finished!


Running set-cover algorithm...


Total cost is  5927212.582298655 


Tableaux: (size = 5)

{'OP_CARRIER_AIRLINE_ID': 'American Airlines Inc.: AA'} 	delta: 227.0  support(%): 24.62
{'OP_CARRIER_AIRLINE_ID': 'United Air Lines Inc.: UA'} 	delta: 320.0  support(%): 12.32
{'OP_CARRIER_AIRLINE_ID': 'Southwest Airlines C